In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/26__walking_down_street_outside_angry.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/08__talking_against_wall.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/14__walking_down_indoor_hall_disgust.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/08__walking_down_street_outside_angry.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/05__outside_talking_still_laughing.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/14__exit_phone_room.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/06__walk_down_hall_angry.mp4
/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences/12__outside_talking_still_laughing.mp4
/kaggle/input/deep-fake-detection-d

In [3]:
!pip install torch torchvision transformers pillow matplotlib opencv-python pandas

In [4]:
!pip install tqdm

In [5]:
# import os
# import cv2
# import torch
# import random
# from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# from transformers import AutoImageProcessor, AutoModelForImageClassification

# # Paths for original and manipulated videos
# original_videos_dir = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences"
# manipulated_videos_dir = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences"

# # Collect video paths and labels
# original_videos = [os.path.join(original_videos_dir, filename) for filename in os.listdir(original_videos_dir)]
# manipulated_videos = [os.path.join(manipulated_videos_dir, filename) for filename in os.listdir(manipulated_videos_dir)]

# original_labels = [0] * len(original_videos)  # 0 for original videos
# manipulated_labels = [1] * len(manipulated_videos)  # 1 for manipulated videos

# all_videos = original_videos + manipulated_videos
# labels = original_labels + manipulated_labels


# class DeepfakeDataset(Dataset):
#     def __init__(self, videos, labels, processor, frame_count=5, transform=None):
#         self.videos = videos
#         self.labels = labels
#         self.processor = processor
#         self.frame_count = frame_count
#         self.transform = transform or transforms.Compose([
#             transforms.Resize((224, 224)),
#             transforms.ToTensor()
#         ])

#     def __len__(self):
#         return len(self.videos)

#     def __getitem__(self, idx):
#         video_path = self.videos[idx]
#         label = self.labels[idx]

        
#     # Extract frames from video
#         cap = cv2.VideoCapture(video_path)
#         frames = []
#         for _ in range(self.frame_count):
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             frame = Image.fromarray(frame)
#             frames.append(self.transform(frame))
#         cap.release()
    
#         # Handle empty frames
#         if len(frames) == 0:
#             # Add default blank frames of size [3, 224, 224]
#             blank_frame = torch.zeros(3, 224, 224)  # RGB with height and width
#             frames = [blank_frame] * self.frame_count
#     # Pad frames if less than required
#         while len(frames) < self.frame_count:
#             frames.append(torch.zeros_like(frames[0]))
    
#         # Stack frames into a tensor and aggregate
#         frames_tensor = torch.stack(frames)
#         aggregated_frame = frames_tensor.mean(dim=0)
    
#         # Ensure the pixel values are within [0, 255]
#         aggregated_frame = aggregated_frame * 255  # Scale to [0, 255]
#         aggregated_frame = aggregated_frame.clamp(0, 255).byte()  # Convert to uint8
    
#         # Process the aggregated frame using the processor
#         inputs = self.processor(images=aggregated_frame, return_tensors="pt", do_rescale=False)
#         pixel_values = inputs['pixel_values'].squeeze(0)
    
#         return pixel_values, torch.tensor(label)



# # Initialize Dataset and DataLoader
# processor = AutoImageProcessor.from_pretrained("Wvolf/ViT_Deepfake_Detection")
# train_videos, val_videos, train_labels, val_labels = train_test_split(
#     all_videos, labels, test_size=0.2, random_state=42, stratify=labels
# )
# train_dataset = DeepfakeDataset(train_videos, train_labels, processor)
# val_dataset = DeepfakeDataset(val_videos, val_labels, processor)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


# # Model Setup
# model = AutoModelForImageClassification.from_pretrained("Wvolf/ViT_Deepfake_Detection")
# model.config.num_labels = 2
# model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# # Training Loop
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
# criterion = torch.nn.CrossEntropyLoss()
# device = model.device


# def evaluate(model, val_loader, criterion):
#     model.eval()
#     val_loss = 0.0
#     val_accuracy = 0.0
#     with torch.no_grad():
#         for pixel_values, labels in val_loader:
#             pixel_values, labels = pixel_values.to(device), labels.to(device)
#             outputs = model(pixel_values=pixel_values)
#             loss = criterion(outputs.logits, labels)
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs.logits, 1)
#             val_accuracy += (predicted == labels).sum().item() / labels.size(0)
#     return val_loss / len(val_loader), val_accuracy / len(val_loader)

# for epoch in range(20):
#     model.train()
#     total_loss = 0.0
#     total_correct = 0
#     total_samples = 0

#     for pixel_values, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
#         pixel_values, labels = pixel_values.to(device), labels.to(device)
#          # Forward pass
#         outputs = model(pixel_values=pixel_values)
#         loss = criterion(outputs.logits, labels)
        
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Update total loss
#         total_loss += loss.item()

#         # Calculate batch accuracy
#         _, predicted = torch.max(outputs.logits, 1)
#         total_correct += (predicted == labels).sum().item()
#         total_samples += labels.size(0)

#     # Calculate average training loss and accuracy
#     avg_train_loss = total_loss / len(train_loader)
#     train_accuracy = total_correct / total_samples

#     # Evaluate on validation data
#     val_loss, val_accuracy = evaluate(model, val_loader, criterion)

#     # Print training and validation metrics
#     print(f"Epoch {epoch+1}: "
#           f"Train Loss = {avg_train_loss:.4f}, Train Acc = {train_accuracy:.4f}, "
#           f"Val Loss = {val_loss:.4f}, Val Acc = {val_accuracy:.4f}")

# # Save the Model
# model.save_pretrained("fine_tuned_deepfake_vit")
# processor.save_pretrained("fine_tuned_deepfake_vit")

In [ ]:
import os
import cv2
import torch
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Paths for original and manipulated videos
original_videos_dir = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences"
manipulated_videos_dir = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences"

# Collect video paths and labels
original_videos = [os.path.join(original_videos_dir, filename) for filename in os.listdir(original_videos_dir)]
manipulated_videos = [os.path.join(manipulated_videos_dir, filename) for filename in os.listdir(manipulated_videos_dir)]

original_labels = [0] * len(original_videos)
manipulated_labels = [1] * len(manipulated_videos)

all_videos = original_videos + manipulated_videos
labels = original_labels + manipulated_labels


class DeepfakeDataset(Dataset):
    def __init__(self, videos, labels, processor, frame_count=8, transform=None):
        self.videos = videos
        self.labels = labels
        self.processor = processor
        self.frame_count = frame_count
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video_path = self.videos[idx]
        label = self.labels[idx]

        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if total_frames > self.frame_count:
            frame_indices = np.linspace(0, total_frames - 1, self.frame_count, dtype=int)
        else:
            frame_indices = range(total_frames)

        frames = []
        for frame_idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame)
            frames.append(self.transform(frame))
        cap.release()

        if len(frames) == 0:
            blank_frame = torch.zeros(3, 224, 224)
            frames = [blank_frame] * self.frame_count

        while len(frames) < self.frame_count:
            frames.append(torch.zeros_like(frames[0]))

        frames_tensor = torch.stack(frames)
        aggregated_frame = frames_tensor.mean(dim=0)

        aggregated_frame = aggregated_frame * 255
        aggregated_frame = aggregated_frame.clamp(0, 255).byte()

        inputs = self.processor(images=aggregated_frame, return_tensors="pt", do_rescale=False)
        pixel_values = inputs['pixel_values'].squeeze(0)

        return pixel_values, torch.tensor(label)


# Load Swin Transformer Model
processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
train_videos, val_videos, train_labels, val_labels = train_test_split(
    all_videos, labels, test_size=0.2, random_state=42, stratify=labels
)

train_dataset = DeepfakeDataset(train_videos, train_labels, processor)
val_dataset = DeepfakeDataset(val_videos, val_labels, processor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Swin Transformer Model
model = AutoModelForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model.config.num_labels = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and Loss Function
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
criterion = torch.nn.CrossEntropyLoss()

# Evaluation Function
def evaluate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    with torch.no_grad():
        for pixel_values, labels in val_loader:
            pixel_values, labels = pixel_values.to(device), labels.to(device)
            outputs = model(pixel_values=pixel_values)
            loss = criterion(outputs.logits, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            val_accuracy += (predicted == labels).sum().item() / labels.size(0)
    return val_loss / len(val_loader), val_accuracy / len(val_loader)

# Training Loop with Mixed Precision
scaler = torch.cuda.amp.GradScaler()

for epoch in range(5):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for pixel_values, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        pixel_values, labels = pixel_values.to(device), labels.to(device)

        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            outputs = model(pixel_values=pixel_values)
            loss = criterion(outputs.logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    val_loss, val_accuracy = evaluate(model, val_loader, criterion)
    scheduler.step()

    print(f"Epoch {epoch+1}: "
          f"Train Loss = {avg_train_loss:.4f}, Train Acc = {train_accuracy:.4f}, "
          f"Val Loss = {val_loss:.4f}, Val Acc = {val_accuracy:.4f}")

# Save Model
model.save_pretrained("fine_tuned_swin_deepfake")
processor.save_pretrained("fine_tuned_swin_deepfake")

<ipython-input-7-a06b466c71dd>:126: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1:   0%|          | 0/86 [00:00<?, ?it/s]<ipython-input-7-a06b466c71dd>:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1:   5%|▍         | 4/86 [06:21<1:48:05, 79.10s/it] 